In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._

import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global

# Monads

## What is a Monad?
A simple definition: Something that can be flatMapped

Another definition: A mechanism for sequencing computations

Functors are also for sequencing computations, but are limited in that they only allow one *complication* to occur at the beginning of the sequence.

In [ ]:
def parseInt(str: String): Option[Int] =
  scala.util.Try(str.toInt).toOption

val some = parseInt("10").map(_ * 5).map(_ + 3)
val none = parseInt("Not an int").map(_ * 5).map(_ + 3)

Monads allow to specify what happens next, taking into account an intermediate *complication*.

In [ ]:
def divide(a: Int, b: Int): Option[Int] =
  if (b == 0) None else Some(a / b)

def stringDivideBy(str1: String, str2: String): Option[Int] =
  parseInt(str1).flatMap { num1 =>
    parseInt(str2).flatMap { num2 =>
      divide(num1, num2)
    }
  }

val some  = stringDivideBy("10", "5")
val none1 = stringDivideBy("Not an int", "5")
val none2 = stringDivideBy("10", "Not an int")
val none3 = stringDivideBy("10", "0")

The *complication* in this example is `Option`s fail-fast error handling behavior.

Every `Monad` is also a `Functor` (we'll do the proof in a bit). If we have `flatMap` and `map` we can use for comprehensions to clarify the sequencing behavior:

In [ ]:
def stringDivideBy(str1: String, str2: String): Option[Int] =
  for {
    num1 <- parseInt(str1)
    num2 <- parseInt(str2)
    ans  <- divide(num1, num2)
  } yield ans

## Examples of Monads

### `List`
`List`'s `flatMap` can be thought of as iterating over the elements of the list. This is reinforced by the syntax of for comprehensions, which look very much like imperative for-loops:

In [ ]:
for {
  x <- (1 to 3).toList
  y <- (4 to 5).toList
} yield (x, y)

However, there is another mental model we can apply that highlights the monadic behaviour of `List`. If we think of `List`s as sets of intermediate results, `flatMap` becomes a construct that calculates permutations and combinations.

`List`'s *complication* is calculating all possible intermediate results.

### `Future`
`Future`'s `flatMap` allows us to sequence asynchronous computations without worrying about the complexities of thread pools and schedulers.

In [ ]:
def doSomethingLongRunning: Future[Int] = ???
def doSomethingElseLongRunning: Future[Int] = ???
def doSomethingVeryLongRunning: Future[Int] =
  for {
    result1 <- doSomethingLongRunning
    result2 <- doSomethingElseLongRunning
  } yield result1 + result2

If you're familiar with `Future`, you’ll know that the code above is running each operation *in sequence*. This becomes clearer if we expand out the for comprehension to show the nested calls to `flatMap`:

In [ ]:
def doSomethingVeryLongRunning: Future[Int] =
  doSomethingLongRunning.flatMap { result1 =>
    doSomethingElseLongRunning.map { result2 =>
      result1 + result2
    }
  }

Each `Future` in our sequence is created by a function that receives the result from a previous `Future`. In other words, each step in our computation can only start once the previous step is finished.



`Future`s *can* be ran in parallel, of course, but monads are all about sequencing computations. Also, running `Future`s in parallel shows how `Future`s break **referential transparency**.

## Definition of a Monad
A monad requires slightly more than just `flatMap` a monad needs both:
- `pure` of type `A => F[A]`
- `flatMap` of type `(F[A], A => F[B]) => F[B]`

`pure` abstracts over constructors. Every monad can be created from a plain value. Think of `pure` as an abstraction of `Some(value)` for `Option`, `Future.successful(value)` for `Future`, and `List(value)` for `List`.

Here is a simplified version of the `Monad` type class in Cats:

In [ ]:
// Calling it MyMonad to not overwrite cats.Monad
trait MyMonad[F[_]] {
  def pure[A](value: A): F[A]
  def flatMap[A, B](value: F[A])(func: A => F[B]): F[B]
}

## Monad Laws
Like most type classes, `Monad` has laws:

**Left Identity**

`pure(value).flatMap(f) == f(value)`

In [ ]:
val result1 = "10".some.flatMap(parseInt)
val result2 = parseInt("10")

**Right Identity**

`monad.flatMap(pure) == monad`

In [ ]:
val result1 = "10".some
val result2 = "10".some.flatMap(_.some)

**Associativity**

`monad.flatMap(f).flatMap(g) == monad.flatMap(x => f(x).flatMap(g))`

In [ ]:
def tenDividedBy(a: Int) = divide(10, a)

val result1 = "10".some.flatMap(parseInt).flatMap(tenDividedBy)
val result2 = "10".some.flatMap(x => parseInt(x).flatMap(tenDividedBy))

## Exercise
Lets do an exercise from the book and prove that every `Monad` is also a `Functor`:

In [ ]:
// Calling it MyMonad to not overwrite cats.Monad
trait MyMonad[F[_]] {
  def pure[A](value: A): F[A]
  def flatMap[A, B](value: F[A])(func: A => F[B]): F[B]
  def map[A, B](value: F[A])(func: A => B): F[B] =
    ???
}

In [ ]:
// Calling it MyMonad to not overwrite cats.Monad
trait MyMonad[F[_]] {
  def pure[A](value: A): F[A]
  def flatMap[A, B](value: F[A])(func: A => F[B]): F[B]
  def map[A, B](value: F[A])(func: A => B): F[B] =
    flatMap(value)(a => pure(func(a)))
}

Side Note:

This interesting phenonemon where we are severly limited in the number of possible implementations of a polymorphic function is called **parametricity**. There is only one way to write this function and have it compile. Go ahead and try to find another.